In [30]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

import os, sys, pathlib, random, time, pickle, copy
from tqdm import tqdm

In [31]:
device = torch.device("cuda:0")
# device = torch.device("cpu")

In [32]:
import dtnnlib

In [33]:
model = dtnnlib.cifar_resnet20(num_classes=10, transform="linear2")
model.to(device)

CifarResNet(
  (conv1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): ChannelNorm2D()
  )
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Sequential(
        (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): ChannelNorm2D()
      )
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Sequential(
        (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): ChannelNorm2D()
      )
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Sequential(
        (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1):

In [34]:
cifar_train = transforms.Compose([
    transforms.RandomCrop(size=32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465], # mean=[0.5071, 0.4865, 0.4409] for cifar100
        std=[0.2023, 0.1994, 0.2010], # std=[0.2009, 0.1984, 0.2023] for cifar100
    ),
])

cifar_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465], # mean=[0.5071, 0.4865, 0.4409] for cifar100
        std=[0.2023, 0.1994, 0.2010], # std=[0.2009, 0.1984, 0.2023] for cifar100
    ),
])

train_dataset = datasets.CIFAR10(root="../../../../_Datasets/cifar10/", train=True, download=True, transform=cifar_train)
test_dataset = datasets.CIFAR10(root="../../../../_Datasets/cifar10/", train=False, download=True, transform=cifar_test)

Files already downloaded and verified
Files already downloaded and verified


In [35]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=128, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=128, shuffle=False, num_workers=2)

In [36]:
for xx, yy in train_loader:
    xx = xx.to(device)
    tt = model(xx)
    print(xx.shape, tt.shape)
    break

torch.Size([128, 3, 32, 32]) torch.Size([128, 10])


In [37]:
yy = model.conv1(xx)
yy.shape

torch.Size([128, 16, 32, 32])

In [38]:
yy = yy-yy.mean(dim=1, keepdim=True)
yy = yy/yy.std(dim=1, keepdim=True)

In [39]:
yy.std(dim=1)

tensor([[[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         ...,
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],

        [[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         ...,
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],

        [[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.

In [40]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

In [41]:
## Following is copied from 
### https://github.com/kuangliu/pytorch-cifar/blob/master/main.py

# Training
def train(epoch):
#     print('\nEpoch: %d' % epoch)
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(tqdm(train_loader)):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    print(f"[Train] {epoch} Loss: {train_loss/(batch_idx+1):.3f} | Acc: {100.*correct/total:.3f} {correct}/{total}")
    return

In [42]:
best_acc = -1
def test(epoch):
    global best_acc
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(tqdm(test_loader)):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
    print(f"[Test] {epoch} Loss: {test_loss/(batch_idx+1):.3f} | Acc: {100.*correct/total:.3f} {correct}/{total}")
    
    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'model': model.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('models'):
            os.mkdir('models')
        torch.save(state, './models/c10_resnet20.pth')
        best_acc = acc

In [43]:
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
resume = False

if resume:
    # Load checkpoint.
    print('==> Resuming from checkpoint..')
    assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
    checkpoint = torch.load('./models/c10_resnet20.pth')
    model.load_state_dict(checkpoint['model'])
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch']

In [44]:
### Train the whole damn thing

for epoch in range(start_epoch, start_epoch+200): ## for 200 epochs
    train(epoch)
    test(epoch)
    scheduler.step()

  0%|          | 0/79 [00:00<?, ?it/s]

[Train] 0 Loss: 1.772 | Acc: 34.782 17391/50000


  0%|          | 0/391 [00:00<?, ?it/s]

[Test] 0 Loss: 1.646 | Acc: 42.100 4210/10000
Saving..


  0%|          | 0/79 [00:00<?, ?it/s]

...

[Test] 198 Loss: 0.325 | Acc: 91.600 9160/10000


  0%|          | 0/79 [00:00<?, ?it/s]

[Train] 199 Loss: 0.023 | Acc: 99.448 49724/50000


100%|██████████| 79/79 [00:01<00:00, 75.24it/s]

[Test] 199 Loss: 0.325 | Acc: 91.610 9161/10000


### Load Best Model State

In [45]:
model = dtnnlib.cifar_resnet20(num_classes=10, transform="linear2")
model.to(device)

CifarResNet(
  (conv1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): ChannelNorm2D()
  )
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Sequential(
        (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): ChannelNorm2D()
      )
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Sequential(
        (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): ChannelNorm2D()
      )
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Sequential(
        (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1):

In [46]:
# model_name = './models/c10_resnet20-distance_199.pth'
# model_name = './models/c10_resnet20-linear_184.pth'
model_name = './models/c10_resnet20.pth'

In [47]:
print('==> Resuming from checkpoint..')
assert os.path.isdir('models'), 'Error: no checkpoint directory found!'
checkpoint = torch.load(model_name)
model.load_state_dict(checkpoint['model'])
best_acc = checkpoint['acc']
start_epoch = checkpoint['epoch']

==> Resuming from checkpoint..


In [48]:
### Linear -> (92.96, 184)
### Distance -> (92.64, 199)
best_acc, start_epoch

(91.63, 197)

In [49]:
!cp ./models/c10_resnet20.pth ./models/c10_resnet20_distance2_.pth

### Benchmark 

In [50]:
import foolbox as fb
import foolbox.attacks as fa

In [21]:
fmodel = fb.PyTorchModel(model.eval(), bounds=(-1, 1))

attack = fb.attacks.LinfPGD()
epsilons = [0.0, 0.001, 0.01, 0.03, 0.1, 0.3, 0.5, 1.0]

In [45]:
images, labels = [], []
for i, (xx, yy) in enumerate(test_loader):
    images.append(xx)
    labels.append(yy)
    if i*128 > 500: break
    
images, labels = torch.cat(images), torch.cat(labels)
images, labels = images.to(device), labels.to(device)

In [46]:
images.shape, labels.shape

(torch.Size([640, 3, 32, 32]), torch.Size([640]))

In [47]:
_, advs, success = attack(fmodel, images, labels, epsilons=epsilons)

In [48]:
len(advs), advs[0].shape

(8, torch.Size([640, 3, 32, 32]))

In [49]:
success.shape

torch.Size([8, 640])

In [50]:
torch.count_nonzero(success)/torch.numel(success)

tensor(0.5162, device='cuda:0')

In [ ]:
### Attack Success

# Bound [0,1]
# 25% success on distance based function
# 30% success on linear function
## linear - 33 on 2176 test points
## distance - 24.14 on 640 test points

# Bound [-1, 1]
# distance - 640 data : 45.33, 45.55
# linear   - 640 data : 54.48, 51.62

## Benchmark on full dataset

In [51]:
attacks = [
    fa.FGSM(),
    fa.LinfPGD(),
    fa.LinfBasicIterativeAttack(),
    fa.LinfAdditiveUniformNoiseAttack(),
#     fa.LinfDeepFoolAttack(),
]
atk_names = [
    "FGSM", 
    "LinfPGD", 
    "LinfBasicIterativeAttack", 
    "LinfAdditiveUniformNoiseAttack", 
#     "LinfDeepFoolAttack",
]

epsilons = [
    0.0,
    0.0005,
    0.001,
    0.0015,
    0.002,
    0.003,
    0.005,
    0.01,
    0.02,
    0.03,
    0.1,
    0.3,
    0.5,
    1.0,
]

In [52]:
def get_attack_success(model, attack, dataloader):
    fmodel = fb.PyTorchModel(model.eval(), bounds=(-1, 1))   
    success_per_eps = []

#     epsilons = [
#     0.0,
#     0.0005,
#     0.001,
#     0.0015,
#     0.002,
#     0.003,
#     0.005,
#     0.01,
#     0.02,
#     0.03,
#     0.1,
#     0.3,
#     0.5,
#     1.0,
# ]
    for eps in epsilons:
        count = 0
        total = 0
        print(f"Running one epoch attack for eps: {eps}")
        for i, (images, labels) in enumerate(tqdm(dataloader)):
            images, labels = images.to(device), labels.to(device)
            _, _, success = attack(fmodel, images, labels, epsilons=[eps])
            count += int(torch.count_nonzero(success))
            total += torch.numel(success)
#             break
        success_per_eps.append(count/total)
    return success_per_eps

In [53]:
# get_attack_success(model, attacks[0], test_loader)

In [61]:
# model_names = ['./models/c10_resnet20-linear_184.pth',
#                 './models/c10_resnet20-distance_199.pth']
# transforms = ['linear', 'distance']
model_names = ['./models/c10_resnet20.pth']
transforms = ['linear2']


outputs = {i:{} for i in transforms}

for i in range(len(transforms)):
    transf = transforms[i]
    model_name = model_names[i]
    print(f"Loading model for {transf}")
    model = dtnnlib.cifar_resnet20(num_classes=10, transform=transf)
    model.to(device)
    
    ### load model and compute attack_success
    print('==> Resuming from checkpoint..')
    assert os.path.isdir('models'), 'Error: no checkpoint directory found!'
    checkpoint = torch.load(model_name)
    model.load_state_dict(checkpoint['model'])
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch']
    
    for j in range(len(attacks)):
        attack = attacks[j]
        atname = atk_names[j]
        print(f"Attacking on model-{transf} using {atname}")
        
        succ_eps = get_attack_success(model, attack, test_loader)
        outputs[transf][atname] = succ_eps
#         break

  0%|          | 0/79 [00:00<?, ?it/s]

Loading model for linear2
==> Resuming from checkpoint..
Attacking on model-linear2 using FGSM
Running one epoch attack for eps: 0.0


  0%|          | 0/79 [00:00<?, ?it/s]

Running one epoch attack for eps: 0.0005


  0%|          | 0/79 [00:00<?, ?it/s]

...

  0%|          | 0/79 [00:00<?, ?it/s]

Running one epoch attack for eps: 0.5


  0%|          | 0/79 [00:00<?, ?it/s]

Running one epoch attack for eps: 1.0


100%|██████████| 79/79 [00:01<00:00, 74.40it/s]


In [62]:
outputs

{'linear2': {'FGSM': [0.0837,
   0.0904,
   0.0976,
   0.1059,
   0.1141,
   0.1309,
   0.1664,
   0.2424,
   0.3468,
   0.4137,
   0.5719,
   0.6771,
   0.7481,
   0.8475],
  'LinfPGD': [0.0837,
   0.0865,
   0.0903,
   0.0932,
   0.0969,
   0.1052,
   0.1249,
   0.1724,
   0.2703,
   0.3474,
   0.5425,
   0.7868,
   0.9281,
   0.9986],
  'LinfBasicIterativeAttack': [0.0837,
   0.087,
   0.0911,
   0.0945,
   0.0999,
   0.1108,
   0.1344,
   0.1944,
   0.3187,
   0.411,
   0.6877,
   0.9011,
   0.9714,
   0.9992],
  'LinfAdditiveUniformNoiseAttack': [0.0837,
   0.0836,
   0.0836,
   0.0839,
   0.0832,
   0.0836,
   0.0838,
   0.0835,
   0.0837,
   0.0842,
   0.0909,
   0.1729,
   0.3062,
   0.6688]}}

In [63]:
import pickle

with open('cifar_adverserial_03.2_v1.pkl', 'wb') as handle:
    pickle.dump(outputs, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('filename.pickle', 'rb') as handle:
#     b = pickle.load(handle)

In [56]:
# with open('./cifar_adverserial_03.2_v0.pkl', 'rb') as handle:
#     b = pickle.load(handle)

In [75]:
b['distance'][atk_names[3]]

[0.0736,
 0.0736,
 0.0734,
 0.0739,
 0.0737,
 0.0734,
 0.0739,
 0.0733,
 0.074,
 0.0735,
 0.0854,
 0.1735,
 0.3371,
 0.7008]

In [74]:
outputs['linear2'][atk_names[3]]

[0.0837,
 0.0836,
 0.0836,
 0.0839,
 0.0832,
 0.0836,
 0.0838,
 0.0835,
 0.0837,
 0.0842,
 0.0909,
 0.1729,
 0.3062,
 0.6688]

In [77]:
a = np.array(b['distance'][atk_names[3]])- np.array(outputs['linear2'][atk_names[3]])
a, a.sum()

(array([-0.0101, -0.01  , -0.0102, -0.01  , -0.0095, -0.0102, -0.0099,
        -0.0102, -0.0097, -0.0107, -0.0055,  0.0006,  0.0309,  0.032 ]),
 -0.04249999999999997)

In [57]:
i=-1

In [58]:
i += 1
print(atk_names[i], "linear vs distance")
print(outputs['linear'][atk_names[i]], outputs['distance'][atk_names[i]])

FGSM linear vs distance


KeyError: 'linear'

In [ ]:
outputs